In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

import requests

import time
from time import sleep
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import unquote

import os
import cx_Oracle
import requests
from io import BytesIO

options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

#네이버 지도 화면의 URL
url = "https://map.naver.com/v5/search"

driver.get(url)

time.sleep(3)


connection = cx_Oracle.connect('kyb/1111@localhost:1521/xe')
cursor = connection.cursor()
cursor.execute('SELECT id, search_name FROM zero')

# id와 search_name을 모두 포함하는 리스트 생성
data_list = [(row[0], row[1]) for row in cursor.fetchall()]
cursor.close()
connection.close()


key_word = '비건카페 달냥'


# 해당 요소 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait


# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경


# 현재 스크래핑된 HTML 요소 스냅샷 만드는 함수
def save_html_snapshot(page_str, file_prefix):
    current_date = datetime.now().strftime('%y%m%d_%H%M%S')

    # 파일 경로 생성
    file_path = f'{file_prefix}_{current_date}.html'

    if not os.path.exists(file_path):
        soup = BeautifulSoup(page_str, 'html.parser')
        # 파일에 저장
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(soup.prettify())


# 검색창 찾기 및 검색어 입력
time_wait(10, 'div.input_box > input.input_search')
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

# 검색 결과가 나올 때까지 대기
sleep(1)
switch_frame('searchIframe')
save_html_snapshot(driver.page_source, "검색_결과")
# 사진 탭 클릭
switch_frame('entryIframe')
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[5]/span').click()  # 사진
sleep(3)
save_html_snapshot(driver.page_source, '사진_iframe')
# 이미지 URL 추출
photo = []
soup = BeautifulSoup(driver.page_source, 'html.parser')
images = soup.find_all("img", class_="K0PDV")  # 모든 사진 요소를 찾습니다.
for img in images:
    image_src = img.get('src')  # 이미지의 URL을 가져옴
    if image_src:
        # URL 디코딩 및 필요한 경우 처리
        decoded_url = image_src.split('&src=')[1] if '&src=' in image_src else image_src
        photo.append(decoded_url)

print(photo[:8])
decoded_url = [unquote(url) for url in photo[:8]]
print(decoded_url)
print(len(decoded_url))
connection = cx_Oracle.connect('kyb/1111@localhost:1521/xe')
cursor = connection.cursor()
# 이미지 데이터를 다운로드하고 데이터베이스에 저장
image_data_list = []
for url in decoded_url:
    response = requests.get(url)
    if response.status_code == 200:
        image_data_list.append(response.content)
    else:
        print("Failed to download image from", url)
        image_data_list.append(None)  # 다운로드 실패시 None 추가

try:
    # 이미지 데이터 삽입
    cursor.execute(f"""
        INSERT INTO zero_images (id, image_id, img1, img2, img3, img4, img5, img6, img7, img8)
        VALUES (20, 20, :img1, :img2, :img3, :img4, :img5, :img6, :img7, :img8)
    """, {
        'img1': image_data_list[0],
        'img2': image_data_list[1],
        'img3': image_data_list[2],
        'img4': image_data_list[3],
        'img5': image_data_list[4],
        'img6': image_data_list[5],
        'img7': image_data_list[6],
        'img8': image_data_list[7]
    })
    connection.commit()

except cx_Oracle.Error as e:
    print("Error occurred:", e)

# 연결 종료
cursor.close()
connection.close()
